In [28]:
import pandas as pd
import numpy as np
import sklearn
import os
from tqdm import tqdm, trange, tqdm_notebook
import re
import nltk
from multiprocessing import cpu_count, Pool
from functools import partial
import ipywidgets
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from rewrite.scorer import score_4class

In [2]:
X_train = pd.read_hdf("X_train_full_allfeatures-NOLABEL.h5", key="df")
y_train = pd.read_hdf("y_train_full.h5", key="df")

In [3]:
y_train.head()

0    unrelated
1        agree
2    unrelated
3    unrelated
4     disagree
Name: Stance, dtype: object

In [4]:
X_train.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,police find mass graves with at least 15 bodie...,"[police, find, mass, graf, with, at, least, 15...",danny boyle is directing the untitled film set...,"[danny, boyle, is, directing, the, untitled, f...",2,0,0,0.014085,...,0,0,0,0,0,0,0,0,0,0
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,hundreds of palestinians flee floods in gaza a...,"[hundred, of, palestinian, flee, flood, in, ga...",hundreds of palestinians were evacuated from t...,"[hundred, of, palestinian, were, evacuated, fr...",10,7,7,0.046083,...,3,0,0,0,0,0,0,0,0,0
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,christian bale passes on role of steve jobs ac...,"[christian, bale, pass, on, role, of, steve, j...",30 year old moscow resident was hospitalized w...,"[30, year, old, moscow, resident, wa, hospital...",5,4,1,0.030303,...,1,0,0,0,0,0,0,0,0,0
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,hbo and apple in talks for 15 month apple tv s...,"[hbo, and, apple, in, talk, for, 15, month, ap...",reuters a canadian soldier was shot at the can...,"[reuters, a, canadian, soldier, wa, shot, at, ...",3,3,0,0.028169,...,1,0,0,0,0,0,0,0,0,0
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",spider burrowed through tourist s stomach and ...,"[spider, burrowed, through, tourist, s, stomac...",fear not arachnophobes the story of bunbury s ...,"[fear, not, arachnophobes, the, story, of, bun...",9,5,4,0.032727,...,4,0,0,0,0,0,0,0,0,0


In [5]:
print(X_train.shape)
print(y_train.shape)

(49972, 49)
(49972,)


In [6]:
def del_str_cols(df): # df should be X, e.g. X_train or X_dev
    del df["articleBody"]
    del df["Headline"]
    for col_name in df.columns:
        if "___" == col_name[0:3]:
            del df[col_name]
#del_str_cols()

In [7]:
Xy_train = pd.concat([X_train, y_train], axis=1)
print(Xy_train.shape)

(49972, 50)


In [8]:
Xy_train.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits,Stance
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,police find mass graves with at least 15 bodie...,"[police, find, mass, graf, with, at, least, 15...",danny boyle is directing the untitled film set...,"[danny, boyle, is, directing, the, untitled, f...",2,0,0,0.014085,...,0,0,0,0,0,0,0,0,0,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,hundreds of palestinians flee floods in gaza a...,"[hundred, of, palestinian, flee, flood, in, ga...",hundreds of palestinians were evacuated from t...,"[hundred, of, palestinian, were, evacuated, fr...",10,7,7,0.046083,...,0,0,0,0,0,0,0,0,0,agree
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,christian bale passes on role of steve jobs ac...,"[christian, bale, pass, on, role, of, steve, j...",30 year old moscow resident was hospitalized w...,"[30, year, old, moscow, resident, wa, hospital...",5,4,1,0.030303,...,0,0,0,0,0,0,0,0,0,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,hbo and apple in talks for 15 month apple tv s...,"[hbo, and, apple, in, talk, for, 15, month, ap...",reuters a canadian soldier was shot at the can...,"[reuters, a, canadian, soldier, wa, shot, at, ...",3,3,0,0.028169,...,0,0,0,0,0,0,0,0,0,unrelated
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",spider burrowed through tourist s stomach and ...,"[spider, burrowed, through, tourist, s, stomac...",fear not arachnophobes the story of bunbury s ...,"[fear, not, arachnophobes, the, story, of, bun...",9,5,4,0.032727,...,0,0,0,0,0,0,0,0,0,disagree


In [9]:
#Xy_train = Xy_train[Xy_train["Stance"] != "unrelated"]

In [10]:
print(Xy_train.shape)

(49972, 50)


In [11]:
y_train = Xy_train["Stance"]
X_train = Xy_train.drop("Stance", axis=1)
print(X_train.shape)
print(y_train.shape)

(49972, 49)
(49972,)


In [12]:
# START SENTIMENT ANALYSIS

In [13]:
tqdm.pandas()

In [14]:
from nltk.sentiment import vader
sid = vader.SentimentIntensityAnalyzer()

/home/gui/anaconda3/envs/nlu4/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [15]:
def vader_polarity_scores(df, text_col_name, col_name_prefix):
    pol_scores = df[text_col_name].progress_apply(lambda hl: pd.Series(sid.polarity_scores(hl)))
    cols = pol_scores.columns
    new_cols = []
    for col_name in cols:
        new_cols.append("vader_"+col_name_prefix+"_"+col_name)
    pol_scores.columns = new_cols
    return pol_scores

vader_hl_df = vader_polarity_scores(X_train, "Headline", "hl")
vader_body_df = vader_polarity_scores(X_train, "articleBody", "body")
X_train = pd.concat([X_train, vader_hl_df, vader_body_df], axis=1)

100%|██████████| 49972/49972 [06:29<00:00, 128.14it/s]


In [16]:
# END SENTIMENT ANALYSIS

In [17]:
del_str_cols(X_train)

In [20]:
print(X_train.shape)
print(y_train.shape)

(49972, 51)
(49972,)


In [21]:
X_train, X_dev, y_train, y_dev = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [22]:
print(X_train.shape)
print(y_train.shape)
print(X_dev.shape)
print(y_dev.shape)

(44974, 51)
(44974,)
(4998, 51)
(4998,)


In [23]:
X_train.head()

,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,wrf_hl_fake,wrf_hl_fraud,wrf_hl_hoax,wrf_hl_false,wrf_hl_deny,wrf_hl_denies,...,chargram_16_early_hits,chargram_16_first_hits,vader_hl_neg,vader_hl_neu,vader_hl_pos,vader_hl_compound,vader_body_neg,vader_body_neu,vader_body_pos,vader_body_compound
44610,9,8,4,0.057377,0,0,0,0,0,0,...,0,0,0.145,0.855,0.0,-0.2960,0.085,0.873,0.042,-0.8658
10499,4,3,1,0.023256,0,0,0,0,0,0,...,0,0,0.000,1.000,0.0,0.0000,0.004,0.955,0.040,0.8325
43957,8,5,4,0.024465,0,0,0,0,0,0,...,0,0,0.062,0.938,0.0,-0.0516,0.141,0.782,0.076,-0.9927
49559,4,2,1,0.009926,0,0,0,0,0,0,...,0,0,0.000,1.000,0.0,0.0000,0.060,0.866,0.074,0.7978
1851,1,1,0,0.008403,0,0,0,0,0,0,...,0,0,0.000,1.000,0.0,0.0000,0.000,0.972,0.028,0.5520


In [24]:
clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)

In [25]:
clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1       43994.9336            1.77m
         2       39359.5278            1.74m
         3       35615.7635            1.72m
         4       32550.1051            1.72m
         5       30021.5762            1.69m
         6       27912.1975            1.69m
         7       26129.6700            1.67m
         8       24628.2887            1.67m
         9       23356.7232            1.67m
        10       22285.5580            1.67m
        20       17050.6981            1.56m
        30       15502.6237            1.44m
        40       14804.4956            1.36m
        50       14355.5579            1.29m
        60       13989.2767            1.23m
        70       13681.9988            1.15m
        80       13405.1976            1.04m
        90       13182.4676           57.94s
       100       12973.7804           53.82s
       200       11321.6110            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=14128, subsample=1.0,
              verbose=True, warm_start=False)

In [26]:
preds = clf.predict(X_dev)

In [27]:
print(classification_report(y_dev, preds))
# With VADER sentiment features

             precision    recall  f1-score   support

      agree       0.70      0.31      0.43       331
   disagree       0.57      0.08      0.14       102
    discuss       0.71      0.86      0.78       876
  unrelated       0.96      0.98      0.97      3689

avg / total       0.89      0.90      0.89      4998



In [31]:
score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

Weighted accuracy: 0.8281232492997199 (1847.75 out of 2231.25)


In [33]:
crf = sklearn.ensemble.RandomForestClassifier(n_estimators=400, n_jobs=8, random_state=42)
crf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=8,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [34]:
preds = crf.predict(X_dev)
print(classification_report(y_dev, preds))

             precision    recall  f1-score   support

      agree       0.75      0.36      0.48       331
   disagree       0.63      0.12      0.20       102
    discuss       0.72      0.87      0.79       876
  unrelated       0.96      0.99      0.97      3689

avg / total       0.90      0.90      0.89      4998



In [35]:
score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

Weighted accuracy: 0.836750700280112 (1867.0 out of 2231.25)


In [ ]:
-----

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
crf = sklearn.ensemble.RandomForestClassifier(n_estimators=200, n_jobs=8, random_state=42)

In [ ]:
crf.fit(X_train, y_train)

In [ ]:
preds = crf.predict(X_dev)
print(classification_report(y_dev, preds))

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
# 4-class problem:
# By guessing most common category (unrelated), we would achieve approx. 73% accuracy. The baseline model achieves 88%.